In [1]:
# ==================================================================================================
# --- Imports
# ==================================================================================================
from study_da.plot import get_title_from_configuration, plot_3D
from study_da.postprocess import aggregate_output_data
import pandas as pd

# ==================================================================================================
# --- Postprocess the study
# ==================================================================================================

df_final = aggregate_output_data(
    "example_3D_scan/tree.yaml",
    l_group_by_parameters=["qx_b1", "qy_b1", "i_oct_b1", "i_oct_b2", "nemitt_x", "nemitt_y"],
    generation_of_interest=2,
    name_output="output_particles.parquet",
    write_output=True,
    only_keep_lost_particles=True,
)

# ==================================================================================================
# --- Plot
# ==================================================================================================


# plot_3D(
#     df_final,
#     "qx_b1",
#     "i_oct_b1",
#     "nemitt_x",
#     "normalized amplitude in xy-plane",
#     xlabel=r"$Q_x$" + "with " + r"$Q_y = Q_x -2 + 0.005$",
#     ylabel="Octupole intensity [A]",
#     z_label=r"Normalized emittance [$\mu$m]",
#     title=title,
#     vmin=4,
#     vmax=6,
#     output_path="output.png",
#     output_path_html="output.html",
#     display_plot=False,
# )


In [2]:
df_final = df_final[["qx_b1", "i_oct_b1", "nemitt_x", "normalized amplitude in xy-plane"]]
df_final = df_final.drop_duplicates()
df_final = df_final.set_index(["qx_b1", "i_oct_b1", "nemitt_x"])
idx = pd.MultiIndex.from_product(
    [df_final.index.levels[0], df_final.index.levels[1], df_final.index.levels[2]]
)
df_final = df_final.reindex(idx, fill_value=None).reset_index()

# Interpolate missing values in df_final for the column "normalized amplitude in xy-plane"
df_final["normalized amplitude in xy-plane"] = df_final.groupby(
    ["qx_b1", "i_oct_b1"]
)["normalized amplitude in xy-plane"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

# Fill remaining missing values with 8
df_final["normalized amplitude in xy-plane"] = df_final["normalized amplitude in xy-plane"].fillna(8)

/tmp/ipykernel_818239/4259091016.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_final["normalized amplitude in xy-plane"] = df_final.groupby(


In [3]:
fig = plot_3D(
    df_final,
    "qx_b1",
    "i_oct_b1",
    "nemitt_x",
    "normalized amplitude in xy-plane",
    xlabel=r'Qx',
    ylabel=r"I [A]",
    z_label=r"Normalized emittance [μm]",
    vmin=4,
    vmax=8,
    output_path="output.png",
    output_path_html="output.html",
    display_plot=False,
)
fig.show(renderer="browser")
